# Assignment A1 [35 marks]



The assignment consists of 4 exercises. Each exercise may contain coding and/or discussion questions.
- Type your **code** in the **code cells** provided below each question.
- For **discussion** questions, use the **Markdown cells** provided below each question, indicated by 📝. Double-click these cells to edit them, and run them to display your Markdown-formatted text. Please refer to the Week 1 tutorial notebook for Markdown syntax.

---
## Question 1: Numerical Linear Algebra [8 marks]

**1.1** Using the method of your choice, solve the linear system $Ax = b$ with

$$ A = \begin{pmatrix}
          1 &  1 & 0 & 1  \\ 
         -1 &  0 & 1 & 1  \\ 
          0 & -1 & 0 & -1  \\ 
          1 & 0 & 1 & 0 
        \end{pmatrix}
        \qquad \text{and} \qquad 
    b = \begin{pmatrix}
           5.2 \cr 0.1 \cr 1.9 \cr 0
        \end{pmatrix},
$$

and compute the residual norm $r = \|Ax-b\|_2$. Display the value of $r$ in a clear and easily readable manner.

**[2 marks]**

In [1]:
import numpy as np #import numpy

def system_solver(A,b):
    '''Define a function that will solve a system of linear equations represented 
    by a matrix A and a vector b. The function will use Gaussian elimination'''
    def row_op(A, alpha, i, beta, j): 
        '''This function applies row operation beta*A_j + alpha*A_i to A_j,
        the jth row of the matrix A. Changes A in place. This function and the
        description is taken from the week04 tutorial for Computing and Numerics.
        '''
        A[j, :] = np.add(beta * A[j, :], alpha * A[i, :])
        
    def switch_nonzero(A ,p , q):
        '''If a element [p,q] in matrix is zero, this function switches row p with one
        below it in which the element in column q is not zero'''
        latch0 = A[p, :].copy #create copy of the relevant row
        latch1 = 0
        latch2 = 0
        for i in A[:,q]: #find first element in relevant column below A[p,q] which is not 0.
            if i != 0 and list(A[:, q]).index(i) > p:
                latch1 = i
                latch2 = list(A[:, q]).index(i)
                break 
        A[p,:]=A[latch2,:] #switch the two rows
        A[latch2,:]=latch0
        

    def RREF(A, b):
        '''This function constructs the augmented matrix (A|b) and applies the necessary
        row operations using row_op() so as to reduce (A|b) to Reduced Row Echelon form. This code
        is an adapted version of REF() function that I wrote during week04 tutorial of Coputing and Numerics'''
        n = len(list(b))
        C = A.copy()
        C = np.c_[A, b] #Concactenate A and b into augmented matrix C
        D_list=[C.copy()] #Make list of Augmented matrix after each for operation (this is fpor reference in part 1.3)
        for i in range(n):
            if C[i,i]==0:
                switch_nonzero(C,i,i)
            row_op(C, 0, 0, 1 / C[i, i], i) #Divide Row i by the element in position [i,i] to get a one on diagonal
            D_list.append(C.copy()) #Append matrix after operation to D_list. Will be used in part 3
            for j in range(i + 1, n): 
                row_op(C, -C[j, i], i, 1, j) #Create zeros below the one on the diagonal for the ith row.
                D_list.append(C.copy()) #Append matrix after operation to D_list
            D=C.copy() #The augmented matrix is now in Row echelon form. Set REF to varibale D (for reference in 1.3)

        for i in reversed(range(n)): #Similar to process above, but this time working upwards in the matrix
            for j in reversed(range(i)):
                row_op(C, -C[j, i], i, 1, j) #Produce zeros above leading one of relevant row
                D_list.append(C.copy()) #We again a copy of C to D_list after each operation
        d = C[:, -1] #d is now vector containing solutions
        C = C[:, :-1] #C is identity matrix
        
        return C,d,D,D_list
    
    return RREF(A,b)

def res_norm(A,b): 
    '''Define res_norm() function which finds residual 2norm.'''
    Ax_b=(A@system_solver(A,b)[1])-b #find vector Ax_b containing Ax-b where x is vector of solutions of linear system
    r=np.sqrt(sum([i**2 for i in Ax_b])) #r is residual norm: square root of the sum of squares of differences Ax-b
    return r

#Define the relevant arrays A and b
A=np.array([[1.0,1.0,0.0,1.0],
           [-1.0,0.0,1.0,1.0], 
           [0.0,-1.0,0.0,-1.0],
           [1.0,0.0,1.0,0.0]])
b=np.array([5.2,0.1,1.9,0.0])

#Print solutions
print(f'The system of solutions is {system_solver(A,b)[1]}')
print()
print(f'Residual norm r={res_norm(A,b)}')

The system of solutions is [  7.1 -16.2  -7.1  14.3]

Residual norm r=3.9511502286300045e-15


**1.2** Repeat the same calculations for the matrix

$$ A = \begin{pmatrix}
          a &  1 & 0 & 1  \\ 
         -1 &  0 & 1 & 1  \\ 
          0 & -1 & 0 & -1  \\ 
          1 & 0 & 1 & 0 
        \end{pmatrix}
        \qquad \text{with} \qquad a \in \{10^{-8}, 10^{-10}, 10^{-12}\}. 
$$

Display the value of $r$ for each value of $a$, and avoid repeating (copy+pasting) code.

**[3 marks]**

In [3]:
a_list=[10**(-8), 10**(-10), 10**(-12)] #Make a list containing the 'a's

for a in a_list: #Iterate over relevant elements of set a to produce relevant residual norms
    A[0,0]=a #change a for each iteration
    print(f'For a={a} the system of solutions is {system_solver(A,b)[1]}') #print the system of solutions
    print(f'For a={a} we have r={res_norm(A,b)}') #print the residual norm
    print() #add some spacing

For a=1e-08 the system of solutions is [ 7.09999984e+08 -1.42000002e+09 -7.10000009e+08  1.42000002e+09]
For a=1e-08 we have r=34.83209261112543

For a=1e-10 the system of solutions is [ 7.09998674e+10 -1.41999977e+11 -7.09999883e+10  1.41999977e+11]
For a=1e-10 we have r=170920.25204869633

For a=1e-12 the system of solutions is [ 7.09958094e+12 -1.41992087e+13 -7.10047098e+12  1.41992087e+13]
For a=1e-12 we have r=1226075882.9361339



**1.3** Summarise and explain your observations in a discussion of no more than $250$ words.

**[3 marks]**

In [ ]:
print('Reference for task 1.3:') 
'''Here I attempt to display some of the errors python does in doing row operations when a=10**(-12)'''

A[0,0]=10**(-12) #Set a 

print('We now show how errors ocurr in the lower right corner of the Augmented Matrix C after certain Row operations')

String_list=['C after Row op 4: R2*10^(-12)','C[2:,3:] after Row op 5: R3+R2 (add 1*R2 to R3)', 
             'C[2:,3:] after Row op 6: R4+10^(12)R2', 'C[2:,3:] after Row op 7: 10^(12)R3',
            'C[2:,3:] after Row op 8: R4-2*R3', 'C[2:,3:] after Row op 9: -1.000055730852182*R4',
            'C[2:,3:] after Row op 10: R3-1.000088900582341*R4'] #Make list of strings that will clarify what row operation I am demonstrating

def error_analysis(s, n): 
    '''Function which prints effect of given row operation in lower right corner of augmented matrix and more accurate
    Python representation of those numbers. In puts are a string from the above list and an integer n referring to  
    matrix after given row operation stored in D '''
    print()
    np.set_printoptions() #Reset number of decimals numpy displays
    print(s)
    if n==5: #If n is 5 we display entire matrix
        for i in system_solver(A,b)[3][5]:
            print(i)
        
    else: #if n not 5 only display lower right corner of matrix
        for i in system_solver(A,b)[3][n][2:,3:]:
            print(i)
    print()
    #Set number of decimal places numpy shows in machine representation of number
    np.set_printoptions(formatter={'float': lambda x: "{0:0.25f}".format(x)}) 
    
    #Now display how python stores relevant numbers in part of matrix just displayed
    if n==5: #if n is 5 show numbers in C[1:,3:]
        print('Accurate to 25 d.p, Python stores C[1:,3:] as:')
        for i in system_solver(A,b)[3][5][1:,3:]:
            for j in i:
                print(np.array([j]))
        
    else: #If n not 5 only show python's representation of numbers in bottom corner
        print('Accurate to 25 d.p. Python stores these as:')
        for i in system_solver(A,b)[3][n][2:,3:]:
            for j in i:
                print(np.array([j]))
    print(20*'-')

    
for h in range(6): #Make error analysis show its results for row operations 4 to 10
    error_analysis(String_list[h], h+5) 
    
    
#Now demonstrate how python stores some numbers
np.set_printoptions(formatter={'float': lambda x: "{0:0.55f}".format(x)}) #Set numpy to display 55 decimal places
print(f'How python stores 0.1: {str(np.array([0.1]))[1:-1]}')
print(f'How python stores 1+10^(-12): {str(np.array([1+10**(-12)]))[1:-1]}')
print()
print(f'Python floats are only guaranteed to be accurate to 15 sig figs: \n0.7500000000000006 == 0.7500000000000005 -->  {0.7500000000000006 == 0.7500000000000005}')

📝 ***Discussion for question 1.3***

(# words without $\LaTeX$ expressions: 246)

Errors arise in solutions because many floating point values are approximated by the binary representation inherent to computers (doc.python.org). As shown above, the value python stores for 0.1 is 0.1000000000000000055511151231257827021181583404541015625 to 55 d.p. Floats are only guaranteed to be accurate to 15 significant figures (docs.python.org). (See last example above).

When multiplying small integers by very large ones, large errors arise because decimals are not exact. Errors accumulate as more Row operations are done. Errors are larger for smaller a because more erroneous numbers (from Python's representation) get magnified.
Consider case when $a=10^{-12}$. Using same method as in code and working by hand on paper we get REF of C:

$\begin{pmatrix}
1 & 10^{12} & 0 & 10^{12} & 5.2*10^{12} \\
0 & 1 & 10^{-12} & 1+10^{-12}& 5.2+10^{-13}\\
0 & 0 & 1 & 1 & 7.1*10^{12}+10^{-1}\\
0 & 0 & 0 & 1 & 1.42*10^{13}+10^{-1}
\end{pmatrix}$

The lower right corner is different from REF created by Python in task 1.2 (see C[2:, 3:] after row op 9).

Noticeable problems arise during 5th row operation on the original augmented matrix. At C[2,3], the result is not precisely $10^{-12}$, but $1.000088900582341e-12$ (see above). This is because Python represents the number $1+10^{-12}$ as $1.0000000000010000889005823$ to 25d.p. After 6th row op the result C[2,3] is $1.0001220703125000$ (to 16 d.p.). 7th op yields $1.0000889005823410$ in C[2,3].
8th op yields $-1.000055730852182$ in C[2,3] and $-14200000000000.098$ in C[3,4]. A big error arises after op 9 yields $14199208666000.83$ in C[3,4]. This value is unchanged for the REF python yields and is very different from that in the real REF (by several millions). 

Errors are magnified as Python iterates up to create zeros above leading 1s. These errors get expressed in residual norms.
 
Sources:
15. Floating Point Arithmetic: Issues and Limitations - Python Documentation
URL:https://docs.python.org/3/tutorial/floatingpoint.html#:~:text=Representation%20error%20refers%20to%20the,exact%20decimal%20number%20you%20expect
Last updated: Feb 17, 2021
Retrieved: Feb 20, 2021

---
## Question 2: Sums [10 marks]

Consider the sum

$$
S_N = \sum_{n=1}^N \frac{2n+1}{n^2(n+1)^2}.
$$

**2.1** Write a function `sum_S()` which takes 2 input arguments, a positive integer `N` and a string `direction`, and computes the sum $S_N$ **iteratively** (i.e. using a loop):
- in the direction of increasing $n$ if `direction` is `'up'`,
- in the direction of decreasing $n$ if `direction` is `'down'`.

For instance, to calculate $S_{10}$ in the direction of decreasing $n$, you would call your function using `sum_S(10, 'down')`.

**[3 marks]**

In [ ]:
def sum_S(N, direction): 
    '''See that each term in the sum above can also be expressed (1/n^2)-(1/((n+1)^2).
    This function calculates the sum above by exploiting this fact. For each n, (1/n^2) is only calculated 
    once so as to reduce number of calculations. Calculating the sum this way is much quicker and yields 
    the same result as I will explain in section 3.3 (I also checked this).'''
    the_sum = 0 #define the sum
    
    if direction == 'up': #conditional if 'up'
        
        first_term = 1 #for n=1 (1/n^2)=1
        for n in range(1, N + 1):
            n_plus_1 = ((1 / (n + 1))**2) #define 1/((n+1)^2 for relevant n
            the_sum += first_term - n_plus_1 #compute (1/n^2)-(1/((n+1)^2) and add to the_sum
            first_term = n_plus_1 #set first_term to 1/((n+1)^2 to prepare for next iteration
        return the_sum, first_term #return the_sum as well as last first term (will be useful for 2.2)
    
    else: #if direction not 'up'. We use reverse process of 'up' method above
        
        second_term = ((1 / (N + 1))**2) #1/((n+1)^2 for biggest n
        for n in reversed(range(1, N + 1)):
            n_term = ((1 / n)**2) #define 1/((n+1)^2 for relevant n
            the_sum += n_term - second_term #add (1/n^2)-(1/((n+1)^2) to the_sum
            second_term = n_term #set second_term to 1/n^2 to prepare for next iteration
            
        return the_sum

**2.2** The sum $S_N$ has the closed-form expression $S_N = 1-\frac{1}{(N+1)^2}$. We assume that we can compute $S_N$ using this expression without significant loss of precision, i.e. that we can use this expression to obtain a "ground truth" value for $S_N$.

Using your function `sum_S()`, compute $S_N$ iteratively in both directions, for 10 different values of $N$, linearly spaced, between $10^3$ and $10^6$ (inclusive).

For each value of $N$, compare the results in each direction with the closed-form expression. Present your results graphically, in a clear and understandable manner.

**[4 marks]**

In [ ]:
up_list = [] #List of sums for 'up' method
down_list = [] #List of sums for 'down' method
closed_list = [] #List of sums using close-form expression
up_err = [] #error of 'up' method
down_err = [] #error of 'down' method


N_list = [int(N) for N in np.linspace(10**3, (10**6), num=10)] #define the N values as specified

for N in N_list: #Loop over the Ns to produce sums for 'up', 'down', and closed form methods
    
    up_sum, first_term = sum_S(N, 'up') #Get the up_sum and term 1//(N+1)^2 embodied by first_term. 
    #Immediately extracting first_term reduces number of necessary calculations of square of very large numbers
    up_list.append(up_sum)#Append 'up' sum
    down_list.append(sum_S(N, 'down')) #Append 'down' sum
    closed_list.append(1 - first_term) #this follows from the definition of the close-form expression
    up_err.append((abs(up_list[-1] - closed_list[-1]))) #We compare methods by examining absolute error. Append absolute error of 'up' method
    down_err.append((abs(down_list[-1] - closed_list[-1]))) #Append error of down method

%matplotlib inline
import matplotlib.pyplot as plt

#Here I make a small table to display errors of 'up' and down methods
print('N-value' + 3 * ' ' + '|Error sum_S(N,"up") to 16 d.p. ' + 
      '|Error sum_S(N,"down") to 16 d.p.')
for i in range(10):
    print(str(N_list[i]) + (10 - len(str(N_list[i]))) * ' ' + '|' +
          str(up_err[i]) + (31 - len(str(up_err[i]))) * ' ' + '|' + str(down_err[i]))

fig,ax=plt.subplots(1,2,figsize=(15, 6)) #construct to subplots next to each other
#In first plot, show sums of each method on log-scale to better demonstrate differences
#In order to more easily construct a proper log-scale, we subtract 1 from all sum values and then reconfigure axis tick labels
ax[0].plot(N_list, [i-1 for i in up_list], 'go-',markersize=10, label='Values using sum_S(N,"up")', alpha=0.5) #Set opacity to 0.5 to be able to differentiate overlapping data-points
ax[0].plot(N_list, [i-1 for i in down_list], 'ro-',markersize=10, label='Values using sum_S(N,"down")', alpha=0.4)
ax[0].plot(N_list, [i-1 for i in closed_list],'b^', label=r'$S_N$')
ax[0].set_ylim(-10**(-5.8), 10**(-11))
ax[0].set_yscale('symlog',linthreshy=10**(-11)) #Define a threshhold within which scale is not log to better see differences between methods
ax[0].set_xticks(N_list)
ax[0].set_xticklabels(N_list,rotation=45)
ax[0].set_yticks([-10**(-i) for i in reversed(range(6,12))]+[0,10**(-11)]) #Define y-scale ticks
ax[0].set_yticklabels([f'1-1e-{i}' for i in reversed(range(6,12))]+[1,'1+1e-11']) #Reconfigure y-ticks to compensate the earlier subtraction of 1
ax[0].set_ylabel(r'$Sum$', fontsize=14)
ax[0].set_xlabel('Value of N', fontsize=14)
ax[0].legend(loc='lower right', fontsize=14)
ax[0].set_title(r'Graph 1: Result of Methods Against N (log scale y-axis)')

#On second subplot demonstrate absolute error of 'up' method against 'down method'
ax[1].plot(N_list, up_err, 'g*-', label='Absolute Error sum_S(N,"up")')
ax[1].plot(N_list, down_err, 'r*-', label='Absolute Error sum_S(N,"down")')
ax[1].set_ylim(-10**(-16), 10**(-11)) #Set limits of y-scale
ax[1].set_yscale('symlog', linthreshy=10**(-15.7)) #Again set threshold within whcih graph is not on log-scale so as to demonstrate error of 'down' method
ax[1].set_xticks(N_list)
ax[1].set_xticklabels(N_list,rotation=45)
ax[1].set_yticks([]) #Reset y_ticks
ax[1].set_yticks([0]+[10**(-i) for i in reversed(range(11,17))]) #make y axis increasing
ax[1].set_yticklabels([0]+[10**(-i) for i in reversed(range(11,17))])
ax[1].set_ylabel(r'Absolute Error from $S_N$', fontsize=14)
ax[1].set_xlabel('Value of N', fontsize=14)
ax[1].legend(loc='center right', fontsize=14)
ax[1].set_title(r'Graph 2: Absolute Error Relative to $S_N$ against N (log scale y-axis)')
plt.show()

**2.3** Describe and explain your findings in no more that $250$ words. Which direction of summation provides the more accurate result? Why?

**[3 marks]**

In [5]:
print('Reference examples for 2.3')
np.set_printoptions(formatter={'float': lambda x: "{0:0.40f}".format(x)}) #Set number of decimal places numpy displays
#show some calculations in numpy:
print(f'Sum of small numbers similar order of magnitude: 10^(-18)+3*10^(-18) = {str(np.array([10**(-18)+3*10**(-18)]))[1:-1]}')
print('----')
print(f'Sum of numbers of very different order of magnitude: 0.1+3*10^(-18)  = {str(np.array([0.1+3*10**(-18)]))[1:-1]}')
print(f'We see that this is precisely the same as what python stores for 0.1 = {str(np.array([0.1]))[1:-1]}')

Reference examples for 2.3
Sum of small numbers similar order of magnitude: 10^(-18)+3*10^(-18) = 0.0000000000000000040000000000000002861697
----
Sum of numbers of very different order of magnitude: 0.1+3*10^(-18)  = 0.1000000000000000055511151231257827021182
We see that this is precisely the same as what python stores for 0.1 = 0.1000000000000000055511151231257827021182


📝 ***Discussion for question 2.3***

(# words: 250)

The 'down' direction provides the more accurate result (graphs and table in 2.2). Floating point values stored by python are accurate only to 15 or 16 significant figures (due to storing in binary). After each addition, Python rounds the resulting sum to a number it can represent correct to 15 or 16 significant figures (Goldberg, 1991). Notice that with 'down' method, Python adds numbers to the_sum in ascending order. Thus, the very small numbers do not get 'rounded away' as they do in the 'up' method, where numbers are added in descending order. With 'down' method, the sum of the very small numbers grows to such a size that when it gets added to the bigger ones, it is not rounded away.

We illustrate this with an example that is shown above. Making numpy use 40 d.p we see that when doing a sum such as $10^{-18}+3*10^{-18}$, python stores the result as $0.0000000000000000040000000000000002861697$, which is accurate to 16 significant figures. 
However, when python does the sum $0.1+3*10^{-18}$ it stores the result as $0.1000000000000000055511151231257827021182$, which is precisely the same as how python stores 0.1.

Obviously, for small enough N, the 'up' method (with which we start by adding 'large' numbers to the_sum) will be relatively accurate since the small numbers do not get so small as to get 'rounded away' once they get added to the_sum.

Additionally, as can be seen from graph 2, small rounding errors can also ocurr for 'down' method since the sum to which numbers are added is growing.

Source: 
'What Every Computer Scientist Should Know About Floating-Point Arithmetic'\\\
Computing Surveys \\\
Goldberg, David \\\
March, 1991 \\\
URL: https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html

---
## Question 3: Numerical Integration [10 marks]

For integer $k \neq 0$ consider the integral 

$$
I(k) = \int_0^{2\pi}  x^4 \cos{k x} \ dx = \frac{32\pi^3}{k^2} - \frac{48\pi}{k^4} \ .
$$

**3.1** Write a function `simpson_I()` which takes 2 input arguments, `k` and `N`, and implements Simpson's rule to compute and return an approximation of $I(k)$, partitioning the interval $[0, 2\pi]$ into $N$ sub-intervals of equal width.

**[2 marks]**

In [ ]:
'''I employ Simpson's composite rule and assume that N should refer to the number of subintervals, each of which 
will have a quadrature calculated on it.'''

def f(x,k): #Define the relevant function
    return (x**4)*np.cos(k*x)

def quadrature(f, xk, wk, a, b, k): #Here I use the quadrature function and docstring from w06 tutorial 
    '''
    Approximates the integral of f over [a, b],
    using the quadrature rule with weights wk
    and nodes xk.
    
    Input:
    f (function): function to integrate (as a Python function object)
    xk (Numpy array): vector containing all nodes
    wk (Numpy array): vector containing all weights
    a (float): left boundary of the interval
    b (float): right boundary of the interval
    
    Returns:
    I_approx (float): the approximate value of the integral
        of f over [a, b], using the quadrature rule.
    '''
    # Define the shifted and scaled nodes
    yk = ((b-a)/2)*(xk+1)+a
    
    # Compute the weighted sum
    I_approx = ((b-a)/2)*(sum(wk*f(yk, k)))
    
    return I_approx

def simpson_I(k,N):
    '''This function should return the approximation for the integral using composite Simpson's rule 
    for a given value of k and when N subintervals are used (the quadrature splitting each subinterval into 2 parts)'''
    
    xk=np.array([-1.,0.,1.]) #Define xk and wk for Simpson's as calculated in w06
    wk=np.array([1/3, 4/3, 1/3])
    
    bounds=np.linspace(0.0,2*np.pi,N+1) #Create an array of N+1 nodes representing bounds for subintervals
    the_sum=0 #Create variable of value zero named the_sum
    
    for i in range(N): #Run quadrature for each subsection contained in bounds and add to the_sum
        the_sum+=quadrature(f,xk,wk,bounds[i],bounds[i+1], k) 

    return the_sum #return the_sum which should now contain our approximation

**3.2** For $k = 1$, and for $\varepsilon \in \{10^{-n} \ |\  n \in \mathbb{N}, 3 \leqslant n \leqslant 8\}$, determine the number $N_{\text{min}}$ of partitions needed to get the value of the integral $I(1)$ correctly to within $\varepsilon$. 

**[2 marks]**

In [ ]:
epsi_set=[10**(-i) for i in range(3,9)] #Define set of epsilons

def I_exact(k): #Define function which calculates exact integral for given k
    return ((32*(np.pi**3)/(k**2))-(48*np.pi/(k**4)))

def N_min_func(k):
    '''This function finds the N_mins values for each epsilon given a certain k. 
    By formula for composite quadrature rules E=alpha*h^r in w06, this function assumes that error 
    is monotonically decreasing for decreasing h (increasing N)'''
    
    N_mins = [0] #Make N_min list with one element 0
    I_ex=I_exact(k) #Define the exact value of integral for the given k
    M_list=[0, 500, 100, 20, 5, 1] #Make list of 'jumps' in N value that function will do to check error at different places
    
    for e in epsi_set: #Iterate over set of epsilons
        M=N_mins[-1] #First, set M to the last element in N_min since N for smaller epsilon is assumed to be larger
        for j in range(1,6): #Iterate over the elements in M_list
            
            M-=M_list[j-1] #Take M_minus previous element in M_list to make sure no values are 'skipped'
            while abs(simpson_I(k, M) - I_ex) > e: #While error is larger, keep adding relevant element in M_list to M
                M+=M_list[j]
            #When abs(simpson_I(k, M) - I_ex) <= e, function will jump back M_list[j-1] places and start making smaller jumps 
            #to check for condition abs(simpson_I(k, M) - I_ex) > e. The last jumps are of size 1 and this is how function find N_min
        
        N_mins.append(M) #Append last value of M to N_mins since this will be smallest M for which abs(simpson_I(k, M) - I_ex) <= e
        
    return N_mins[1:] #Return N_mins without the starting 0

print(f'N_min for k=1 (each successive element corresponds to tenfold decrease epsilon): \n{N_min_func(1)}')

**3.3** Repeat your calculations from **3.2** for $k \in \{2^{n}\ |\ n \in \mathbb{N}, n \leqslant 6\}$. 

**[2 marks]**

In [ ]:
A=[N_min_func(1)] #Initiate list to store N_mins for each value of k

ks=[1,2,4,8,16,32,64] #list of k's

for k in ks[1:]: #Iterate over k-values
    mins=N_min_func(k) #Define mins so that N_min_func(k) does not have to be calculated twice
    
    print(f'for k = {k} we have N_min = {mins} for e in epsi_set, respectively') #display results
    
    A.append(mins) #Append N_mins for each k to A
    
A=np.array(A) #make A an np.array

**3.3** Present your results graphically by plotting 

(a) the number of terms $N_{\text{min}}$ against $\varepsilon$ for fixed $k$, 

(b) the number of terms $N_{\text{min}}$ against $k$ for fixed $\varepsilon$.

You should format the plots so that the data presentation is clear and easy to understand.

**[2 marks]**

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(20, 10)) #Create two subplots
plot_types=['g^-', 'b^-', 'r^-', 'k^-', 'c^-', 'm^-', 'y^-'] #list of plot types that I will use in plot (color, marker, and line)

#In first subplot we plot reversed list of epsilons against the relevant N_mins as stored in rows of A
#In other words we plot epsilon against the minimum N such that |approximation - exact integral|<epsilon for constant k

for i in range(7): #Loop over range 0-6 to plot reversed rows in A (in log10) against reversed epsi_set (in log10). 
    ax[0].plot(np.log10(epsi_set[::-1]),np.log10(A[i][::-1]), plot_types[i], markersize=5, label=r'$\log_{10}(N_{min})$ vs $\log_{10}({\varepsilon})$ ' + f'for k={ks[i]}') 
    #note that I have reversed epsi_set so that log10(epsilon) is growing along the x-axis
    
#Specify Title, axis labels, and location + size of legend for Graph 1:
ax[0].set_ylabel(r'$\log_{10}(N_{min})$', fontsize=16)
ax[0].set_xlabel(r'$\log_{10}({\varepsilon})$', fontsize=17)
ax[0].legend(loc='upper right', fontsize=13)
ax[0].set_title(r'Graph 1: $\log_{10}(N_{min})$ against $\log_{10}({\varepsilon})$ for fixed $k$', fontsize=18)

#In second subplot we plot ks against relevant N_mins as stored in columns of B
#In other words we plot k against the minimum N such that |approximation - exact integral|<epsilon for constant epsilon

for i in range(6): #Loop over range 0-6 to plot columns in A (in log2 and starting at last column) against ks.
    ax[1].plot(np.log2(ks), np.log2(A[:,5-i]), plot_types[i], markersize=5, label=r'$\log_{10}(N_{min})$ vs $\log_{2}({k})$ for $\varepsilon$=' + f'1e-{8-i}')

#Specify Title, axis labels, and location + size of legend for Graph 2:
ax[1].set_ylabel(r'$\log_{2}(N_{min})$', fontsize=16)
ax[1].set_xlabel(r'$\log_{2}({k})$', fontsize=14)
ax[1].legend(loc='upper left', fontsize=13)
ax[1].set_title(r'Graph 2: $\log_{10}(N_{min})$ against $\log_{2}({k})$ for fixed $\varepsilon$', fontsize=18)

plt.show() #show graphs

**3.4** Discuss, with reference to your plot from 3.3, your results. Your answer should be no more than $250$ words.

**[2 marks]**

In [ ]:
print('References for task 3.3:')
np.set_printoptions() #reset number of decimal places numpy displays

#Compute and print coefficients for lines of best fit for data points in Graph 1 (keeping k constant). 
#We do this using the inbuilt numpy function polyfit, which returns the coefficients for line of best fit (highest degree first)
#Documentation for polyfit that I consulted is here: https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html
print(r'Coefficients of epsilon in line of best fit for data points in Graph 1:')
print()
coeff1_sum=0 #create as sum of coefficients used for taking average later

#Use polyfit with log10(error) (error stored in reversed epsi_set) as independent variable 
#and log10(N_min) (stored in reversed rows of A) as dependent variable 
#Do this for each row of A:
for i in range(7):
    coefficient=np.polyfit(np.log10(epsi_set[::-1]), np.log10(A[i][::-1]),  1)[0] 
    print(r'Coefficient of epsilon ' + f'for k={ks[i]}: ' + str(coefficient))
    coeff1_sum+=coefficient #add to the sum of coefficients
print(20*'-')
print()

#Compute and print coefficients for lines of best fit for data points in Graph 2 (keeping epsilon constant):
print(r'Coefficients of k in line of best fit for data points in Graph 2:')
print()
coeff2_sum=0 #create as sum of coefficients used for taking average later

#Use polyfit with log2(k) (k stored in ks) as independent variable 
#and log2(N_min) (N_min stored in columns of A) as dependent variable 
#Do this for each column of A:
for i in range(6):
    coefficient = np.polyfit(np.log2(ks), np.log2(np.array(A)[:,i]), 1)[0]
    print(r'Coefficient of k ' + f'for epsilon={epsi_set[i]}: ' + str(coefficient))
    coeff2_sum+=coefficient #add to the sum of coefficients
print(20*'-')
print()

#Print average of coefficients for each of the above:
print(f'Average of coefficients Graph 1: {coeff1_sum/7}')
print(f'Average of coefficients Graph 2: {coeff2_sum/6}')

📝 ***Discussion for question 3.4***

(# words without $\LaTeX$ expressions: 248)

For constant k, there is an approximately linear relationship between $\log_{10}{\varepsilon}$ and $\log_{10}{N_{min}}$ (graph 1). This is expressed in formula describing the error of composite quadrature rules (tutorial w06)
$$
\varepsilon = \alpha h^r = \alpha (\frac{1}{N})^r,
$$
which, when taking $\log_{10}$ on both sides, rearranges to
$$
\frac{-\log_{10}{\varepsilon} + \log_{10}{\alpha}}{r} = \log_{10}({N}).
$$

$$
$$

Since each $N_{min}$ is smallest N such that approximation error < $\varepsilon$, consecutive data-points in Graph 1 do $not$ correspond to a decrease in N such that error of approximation increases $precisely$ tenfold. However, by assuming it does, we can apply above formula if we remember that results will have some small error.


Using numpy's polyfit module, for constant k, the coefficient of the line of best fit between $\log_{10}{\varepsilon}$ and $\log_{10}{N_{min}}$ is always close to -0.249 (see above). Average of coefficients gives $\approx -0.2488641$. Treating $\log{\alpha}$ as a constant and using the formula:
$$$$
$$
\log_{2}({N_2}) - \log_{2}({N_1}) = \frac{-\log_{10}{\varepsilon_1} - 1 + \log_{10}{\alpha}}{r} - \frac{-\log_{10}{\varepsilon_1} + \log_{10}{\alpha}}{r} \approx -0.2488641
$$
$$$$
$$
\Rightarrow -0.2488641 \approx \frac{-1}{r},
$$
and thus
$$
r \approx 4.018
$$
as approximate (to 4 d.p) rate of convergence of composite Simpson rule that we can extrapolate from data.

Graph 2 shows an approximate linear relationship between $\log_{2}{k}$ and $\log_{2}{N_{min}}$. Using polyfit (and keeping in mind the ineherent error as mentioned above), we get a coefficient always close to 0.502. Average $\approx 0.502073129$. In the above equation, changes to k must get expressed in $\alpha$.
Thus, treating $\varepsilon$ as constant, and considering change of 1 in $\log_{2}{k}$
$$$$
$$
\log_{2}({N_2}) - \log_{2}({N_1}) = \frac{-\log_{2}{\varepsilon} + \log_{2}{\alpha_1} + \Delta\log_{2}{\alpha}}{r} - \frac{-\log_{2}{\varepsilon} + \log_{2}{\alpha_1}}{r} \approx 0.502073129
$$
$$$$
$$
\Rightarrow \frac{\Delta\log_{2}{\alpha}}{r} \approx 0.502073129
$$
and thus
$$
\Delta\log_{2}{\alpha} \approx 2.017
$$ to 4 d.p. Written differently,
$$
\log_{2}{\alpha} \approx 2.017\log_{2}{k} + C
$$
$$$$
$$
\Rightarrow \alpha \approx 2^C*k^{2.017}
$$
$$$$
The reason that larger $N_{min}$s result for larger $k$s for same $\varepsilon$ is that changes in x cause 'quicker' changes in $x^4\cos(kx)$. Thus, the interpolating polynomials of degree two within Simpson's rule become less accurate (relative to actual graph). Since k changes at constant relative rate, the graph compresses at constant relative rate, leading to constant relative rate change in $\alpha$.

---
## Question 4: Numerical Derivatives [7 marks]

Derivatives can be approximated by finite differences in several ways, for instance

\begin{align*}
        \frac{df}{dx} & \approx \frac{f(x+h) - f(x)}{h} \\
        \frac{df}{dx} & \approx \frac{f(x) - f(x-h)}{h}  \\
        \frac{df}{dx} & \approx \frac{f(x+h) - f(x-h)}{2h} \ . 
\end{align*}

Assuming $f$ to be differentiable, in the limit $h \to 0$, all three expressions are equivalent and exact, but for finite $h$ there are differences. Further discrepancies also arise when using finite precision arithmetic.

**4.1**
Estimate numerically the derivative of $f(x) = \cos(x)$ at $x = 1$ using the three expressions given above and different step sizes $h$. Use at least 50 logarithmically spaced values $h \in [10^{-16}, 10^{-1}]$.

**[2 marks]**

In [ ]:
h=np.logspace(-16,-1,num=100) #create array of 100 logarithmically spaced values in specified range

est1_list=(np.cos(1+h)-np.cos(1))/h #create an array of values containing forward derivative approximations for each h
est2_list=(np.cos(1)-np.cos(1-h))/h #create an array of values containing backward derivative approximations for each h
est3_list=(np.cos(1+h)-np.cos(1-h))/(2*h) #create an array of values containing centered derivative approximations for each h

**4.2**
Display the absolute difference between the numerical results and the
exact value of the derivative, against $h$ in a doubly logarithmic plot. 
You should format the plot so that the data presentation is clear and easy to understand.

**[2 marks]**

In [ ]:
#Note that (d/dx)cos(x))-sin(x)

err1=abs(est1_list+np.sin(1)) #Compute absolute error for forward approximation
err2=abs(est2_list+np.sin(1)) #Compute absolute error for backward approximation
err3=abs(est3_list+np.sin(1)) #Compute absolute error for centered approximation

fig,ax=plt.subplots(1,1,figsize=(20, 10)) #Create one (sub)plot
#Plot the h-values against the absolute errors for each method
ax.plot(h,err1,'g^-',markersize=5, label=r'$\frac{df}{dx} \approx \frac{f(x+h)-f(x)}{h}$ error against h')
ax.plot(h,err2,'b^-',markersize=5, label=r'$\frac{df}{dx} \approx \frac{f(x)-f(x-h)}{h}$  error against h')
ax.plot(h,err3,'r^-',markersize=5, label=r'$\frac{df}{dx} \approx \frac{f(x+h)-f(x-h)}{2h}$  error against h')
ax.set_yscale('log',basey=10) #Set log-log scale with y-scale in base 10
ax.set_xscale('log')
ax.set_xticks([10**(-i) for i in range(0,17)])
ax.set_yticks([10**(-i) for i in range(1,12,2)])
ax.set_xticklabels([10**(-i) for i in range(0,17)],rotation=45, fontsize=14)
ax.set_yticklabels([10**(-i) for i in range(1,12,2)], fontsize=14)
ax.set_ylabel('Error of Approximation', fontsize=18)
ax.set_xlabel(r'$h$', fontsize=18)
ax.set_title(r'Absolute error of approximations against h', fontsize=28)
ax.legend(loc='lower left', fontsize=22)
ax.minorticks_off() #Turn off annoying small ticks on y-axis
plt.show()

**4.3**
Describe and interpret your results in no more than 250 words.

*Hint: run the code below.*

**[3 marks]**

In [ ]:
h = 1e-15
print(1 + h - 1)
print((1 + h - 1)/h)

📝 ***Discussion for question 4.3***

(# words without $\LaTeX$ expressions: 246)

For $h < 10^{-8}$ the computed approximations display similar accuracy. Absolute errors of the shown magnitudes arise for $h < 10^{-8}$ because Python floats are only guaranteed accurate 15 significant figures (see task 1.3). Python uses its (binary) representation of numbers to compute sums (or differences) and then rounds the result to a number it can represent. For any $h = 10^{-x}$, imprecise numbers get amplified by $h = 10^{x}$, as numerators are divided by $h$ and $2h$. As h increases, less imprecise digits get amplified.

Forward ($D_1(x)$) and backward ($D_{-1}(x)$) approximations are most accurate for $h \approx 10^{-8}$ since they are both first order accurate (as can be dervied from tutorial 7):
\begin{equation}
  D_1(x) - F' \left( x \right)
  = \frac{\Delta x}{2}  F'' \left( x \right) + \frac{\Delta x^2}{6} F'''\left(x\right) + \dots = O(\Delta x)
\end{equation}

\begin{equation}
  D_{-1}(x) - F' \left( x \right)
  = \frac{-\Delta x}{2}  F'' \left( x \right) + \frac{\Delta x^2}{6} F'''\left(x\right) + \dots = O(\Delta x)
\end{equation}

For $h \approx 10^{-8}$, rounding errors only magnify imprecise figures from Python's decimal representation by $\approx 10^{8}$ and the forward and backward approximation formulae are accurate to $\approx 10^{-8}$. As $h > 10^{-8}$, forward and backward approximations become less accurate at a (log-log) linear rate due to their error expressions. Although their error seemingly coincide for $h > 10^{-7}$, there are still small differences in their (written out) error expressions.

$$
$$

The central difference approximation is the average of the other methods. For $h < 10^{-8}$, it suffers from error for the same reasons as the others. However, for $10^{-8} < h < 10^{-5}$ the central method is more accurate due to its second order accuracy (derived from formulae above):

\begin{equation}
  D_C(x) - F' \left( x \right)
  = \frac{\Delta x^2}{6} F'''\left(x\right) + \frac{\Delta x^4}{120} F'''''\left(x\right) + \dots = O(\Delta x^2)
\end{equation}

For $10^{-8} < h < 10^{-5}$, the central diff. method's error is very close to zero and imprecise numbers from Python's decimal representation are decreasingly amplified. For $h > 10^{-4}$, however, the error increases log-log linearly.  

